# Análisis textual de La Voz del Sur y Letra Fría

## Data cleaning La Voz del Sur

In [ ]:
# pd.read_csv is for .cvs files, but this one is Excel so it's pd.read_excel, followed by the file route.
# When reading Excel files I usually open them first in Excel to get familiar with the data structure.
# After commas we can add additional instructions, such as the sheet name. We are not using the first two rows, so we are skipping them.

# Loading every file as separate dataframes. I'm identifying them with distinct names, but we will be combining them into a single larger dataframe in a bit.
#La Voz del Sur
dfvoz = pd.read_csv('scraped data/articulos_lavozdelsur.csv')

In [ ]:
# First we import pandas, our library to work with data sets
import pandas as pd
# This below it's optional, but I like to add it so it shows me all columns in the preview, specially useful for wide tables
pd.options.display.max_columns = None

# This below are optional too but I don't think I'll be using it
#remove scientific notation
#pd.options.display.float_format = '{:.2f}'.format

import re

In [ ]:
# Eliminar duplicados (conservando primeros)
voz = dfvoz.drop_duplicates()
voz

,titulo,articulo,fecha,url
0,Dan Banderazo de Arranque al Operativo de Sema...,"Published on abril 12th, 2025 | by lavozsur\n\...","abril 12th, 2025",https://www.lavozdelsur.com.mx/dan-banderazo-d...
1,La Falta de Vigilancia en las Carreteras,"Published on abril 11th, 2025 | by lavozsur\n\...","abril 11th, 2025",https://www.lavozdelsur.com.mx/la-falta-de-vig...
2,Banda Real de la Montaña Lleva su Música al Fe...,"Published on abril 11th, 2025 | by lavozsur\n\...","abril 11th, 2025",https://www.lavozdelsur.com.mx/banda-real-de-l...
3,Muere un Hombre Atropellado en la Autopista Gu...,"Published on abril 10th, 2025 | by lavozsur\n\...","abril 10th, 2025",https://www.lavozdelsur.com.mx/muere-un-hombre...
4,Conforman Ayuntamiento Infantil 2025 Frente Au...,"Published on abril 9th, 2025 | by lavozsur\n\n...","abril 9th, 2025",https://www.lavozdelsur.com.mx/conforman-ayunt...
...,...,...,...,...
1051,"Toluca, Real Huescalapa y Rayos de Zapotiltic ...","Published on diciembre 27th, 2023 | by lavozsu...","diciembre 27th, 2023",https://www.lavozdelsur.com.mx/toluca-real-hue...
1052,Abren Nevado a Visitantes Para Disfrutar el Pa...,"Published on diciembre 27th, 2023 | by lavozsu...","diciembre 27th, 2023",https://www.lavozdelsur.com.mx/abren-nevado-a-...
1053,Pablo Lemus Lleva Amplia Ventaja Sobre Otros P...,"Published on diciembre 26th, 2023 | by lavozsu...","diciembre 26th, 2023",https://www.lavozdelsur.com.mx/pablo-lemus-lle...
1054,"Bajío, San José y Nuevos Valores Triunfan en l...","Published on diciembre 26th, 2023 | by lavozsu...","diciembre 26th, 2023",https://www.lavozdelsur.com.mx/bajio-san-jose-...


In [ ]:
# Función para extraer los tags y limpiar el artículo
def procesar_articulo(texto):
    # Buscar los tags (ejemplo: "Tags: Chava Zamora, jalisco, ...")
    tags_match = re.search(r'Tags:\s*(.*?)(?=\n\n|$)', texto, flags=re.DOTALL | re.IGNORECASE)
    tags = tags_match.group(1).strip() if tags_match else ""

    # Eliminar la línea de tags del artículo
    texto_limpio = re.sub(r'Tags:.*?(?=\n\n|$)', '', texto, flags=re.DOTALL | re.IGNORECASE)

    # Limpieza adicional (lo que ya teníamos antes)
    texto_limpio = re.sub(r'Published on.*?by lavozsur', '', texto_limpio, flags=re.DOTALL)
    texto_limpio = re.sub(r'lavozsur Con 35 años de trayectoria.*?Jalisco\.', '', texto_limpio, flags=re.DOTALL)
    texto_limpio = re.sub(r'Comments are closed\.', '', texto_limpio, flags=re.DOTALL)
    texto_limpio = texto_limpio.strip()

    return texto_limpio, tags

# Aplicar la función y crear las nuevas columnas
voz[['articulo', 'tags']] = pd.DataFrame(voz['articulo'].apply(procesar_articulo).tolist(), index=voz.index)

# Reordenar columnas (opcional)
voz = voz[['titulo', 'articulo', 'tags', 'fecha', 'url']]
# Guardar el resultado
#voz.to_csv('vozdelsur_limpio_con_tags.csv', index=False)

In [ ]:
# Función para extraer el lugar y limpiar el artículo
def extraer_lugar(texto):
    # Patrón para ciudades en mayúsculas seguidas de "JAL." o similares
    lugar_match = re.search(r'^([A-ZÁÉÍÓÚÜÑ\s]+(?:,\s*JAL\.?|\.))', texto)
    if lugar_match:
        lugar = lugar_match.group(1).strip()
        texto_limpio = texto.replace(lugar, '', 1).strip()
    else:
        lugar = ""
        texto_limpio = texto
    return texto_limpio, lugar

# Aplicar la función para extraer lugares
voz[['articulo', 'lugar']] = pd.DataFrame(voz['articulo'].apply(extraer_lugar).tolist(), index=voz.index)

# Función para verificar si contiene (ES)
def contiene_ES(texto):
    return 'yes' if '(ES)' in texto else 'no'

# Crear columna 'contiene_ES'
voz['contiene_ES'] = voz['articulo'].apply(contiene_ES)

# Reordenar columnas (opcional)
column_order = ['titulo', 'articulo', 'lugar', 'tags', 'contiene_ES', 'fecha', 'url']
voz = voz[column_order]

# Guardar el resultado
voz.to_csv('cleaned_vozdelsur.csv', index=False, encoding='utf-8-sig')

In [ ]:
voz

,titulo,articulo,lugar,tags,contiene_ES,fecha,url
0,Dan Banderazo de Arranque al Operativo de Sema...,*Autoridades de los tres niveles de gobierno t...,,"Chava Zamora, jalisco, la voz del sur de jalis...",no,"abril 12th, 2025",https://www.lavozdelsur.com.mx/dan-banderazo-d...
1,La Falta de Vigilancia en las Carreteras,"Sin duda alguna, los recientes cierres en las ...",,"autopistas, Ciudad Guzmán, Falta de Agua, Oper...",no,"abril 11th, 2025",https://www.lavozdelsur.com.mx/la-falta-de-vig...
2,Banda Real de la Montaña Lleva su Música al Fe...,Las potentes y muy alegres notas del estilo mu...,,"Cultura, La Vos del Sur, Múscia Regional Mexic...",no,"abril 11th, 2025",https://www.lavozdelsur.com.mx/banda-real-de-l...
3,Muere un Hombre Atropellado en la Autopista Gu...,", (ES).- Intensa movilización de unidades de e...","GÓMEZ FARÍAS, JAL.","autopista, Ciuadad Guzmán, GómezFarías, muerto...",yes,"abril 10th, 2025",https://www.lavozdelsur.com.mx/muere-un-hombre...
4,Conforman Ayuntamiento Infantil 2025 Frente Au...,(ES).- El Gobierno Municipal de Zapotlán el Gr...,"CIUDAD GUZMÁN, JAL.","Ayuntamiento Infantil de Zapotlán, Cabildo Inf...",yes,"abril 9th, 2025",https://www.lavozdelsur.com.mx/conforman-ayunt...
...,...,...,...,...,...,...,...
1051,"Toluca, Real Huescalapa y Rayos de Zapotiltic ...",", (ES).- En la jornada 9 en la Liga de Fútbol ...","CIUDAD GUZMÁN, JAL.","Ciudad Guzmán, deportes, futbol, huescalapa, I...",yes,"diciembre 27th, 2023",https://www.lavozdelsur.com.mx/toluca-real-hue...
1052,Abren Nevado a Visitantes Para Disfrutar el Pa...,*Habrá Restricciones Para Vehículos Compactos\...,,"Ciudad Guzmán, colima, la voz del sur de jalis...",yes,"diciembre 27th, 2023",https://www.lavozdelsur.com.mx/abren-nevado-a-...
1053,Pablo Lemus Lleva Amplia Ventaja Sobre Otros P...,Durante el 9 y 13 de diciembre la empresa Tych...,,"Ciudad Guzmán, Elecciones 2024, encuestas, Gub...",no,"diciembre 26th, 2023",https://www.lavozdelsur.com.mx/pablo-lemus-lle...
1054,"Bajío, San José y Nuevos Valores Triunfan en l...",", (ES).- En el fútbol de la Categoría los equi...","CIUDAD GUZMÁN, JAL.","Bajío, Categoría Juvenil, Ciudad Guzmán, Club ...",yes,"diciembre 26th, 2023",https://www.lavozdelsur.com.mx/bajio-san-jose-...


## Letra fría

In [ ]:
# Loading every file as separate dataframes. I'm identifying them with distinct names, but we will be combining them into a single larger dataframe in a bit.
#La Voz del Sur
dfletra = pd.read_csv('scraped data/letra_fria_200_20250414_120843.csv')
dfletra

,titulo,articulo,fecha,autor,url,categoria
0,Prohíben acceso de vehículos a playas de Melaque,"Cihuatlán, Jalisco.- La delegación municipal d...",NaN,NaN,https://letrafria.com/prohiben-acceso-de-vehic...,Región
1,Abrirán módulo permanente de licencias de cond...,"El Grullo, Jalisco.- A través de una gestión r...",NaN,NaN,https://letrafria.com/abriran-modulo-permanent...,Región
2,"Tras protestas, acuerdan mesas de trabajo por ...","Guadalajara, Jalisco.- Estudiantes de la Unive...",NaN,NaN,https://letrafria.com/tras-protestas-acuerdan-...,Región
3,Encuentro Jóvenes y Agroecología en El Limón s...,"El Limón, Jalisco.- El Encuentro Jóvenes y Agr...",NaN,NaN,https://letrafria.com/encuentro-jovenes-y-agro...,Región
4,Sandra Borja,"Mientras vivimos tiempos de cuaresma, de “guar...",NaN,NaN,https://letrafria.com/sandra-borja/,Región
...,...,...,...,...,...,...
1393,Unión de Tvla alista todo para vivir el Festiv...,"Por: Vianney Martínez Pérez\n\nUnión de Tvla, ...",NaN,NaN,https://letrafria.com/union-de-tvla-alista-tod...,Región
1394,Prohíben acceso de vehículos a playas de Melaque,"Cihuatlán, Jalisco.- La delegación municipal d...",NaN,NaN,https://letrafria.com/prohiben-acceso-de-vehic...,Región
1395,Abrirán módulo permanente de licencias de cond...,"El Grullo, Jalisco.- A través de una gestión r...",NaN,NaN,https://letrafria.com/abriran-modulo-permanent...,Región
1396,Encuentro Jóvenes y Agroecología en El Limón s...,"El Limón, Jalisco.- El Encuentro Jóvenes y Agr...",NaN,NaN,https://letrafria.com/encuentro-jovenes-y-agro...,Región


In [ ]:
# Eliminar duplicados (conservando primeros)
letra = dfletra.drop_duplicates()
letra

,titulo,articulo,fecha,autor,url,categoria
0,Prohíben acceso de vehículos a playas de Melaque,"Cihuatlán, Jalisco.- La delegación municipal d...",NaN,NaN,https://letrafria.com/prohiben-acceso-de-vehic...,Región
1,Abrirán módulo permanente de licencias de cond...,"El Grullo, Jalisco.- A través de una gestión r...",NaN,NaN,https://letrafria.com/abriran-modulo-permanent...,Región
2,"Tras protestas, acuerdan mesas de trabajo por ...","Guadalajara, Jalisco.- Estudiantes de la Unive...",NaN,NaN,https://letrafria.com/tras-protestas-acuerdan-...,Región
3,Encuentro Jóvenes y Agroecología en El Limón s...,"El Limón, Jalisco.- El Encuentro Jóvenes y Agr...",NaN,NaN,https://letrafria.com/encuentro-jovenes-y-agro...,Región
4,Sandra Borja,"Mientras vivimos tiempos de cuaresma, de “guar...",NaN,NaN,https://letrafria.com/sandra-borja/,Región
...,...,...,...,...,...,...
1385,Hay menos pobres en Jalisco: Coneval,"Por: Lauro Rodríguez.\n\nGuadalajara, Jalisco....",NaN,NaN,https://letrafria.com/hay-menos-pobres-en-jali...,Región
1386,"Itei impone amonestaciones, otra vez, a funcio...","Por: Lauro Rodríguez.\n\nTomatlán, Jalisco.- E...",NaN,NaN,https://letrafria.com/itei-impone-amonestacion...,Región
1391,Romería 2023: La virgen peregrina regresa a At...,"Por: Vianney Martínez Pérez\n\nAtengo, Jalisco...",NaN,NaN,https://letrafria.com/romeria-2023-la-virgen-p...,Región
1392,Todo listo para celebrar el Día de las Paseado...,"Por: Vianney Martínez Pérez\n\nLa Huerta, Jali...",NaN,NaN,https://letrafria.com/todo-listo-para-celebrar...,Región


In [ ]:
# Función para extraer los bylines y limpiar el artículo
def procesar_articulo(texto):
    # Buscar los bylines (ejemplo: "Por: Lauro Rodríguez, 	Por: Vianney Martínez Pérez, etc")
    tags_match = re.search(r'Por:\s*(.*?)(?=\n\n|$)', texto, flags=re.DOTALL | re.IGNORECASE)
    tags = tags_match.group(1).strip() if tags_match else ""

    # Eliminar la línea de tags del artículo
    texto_limpio = re.sub(r'Por:.*?(?=\n\n|$)', '', texto, flags=re.DOTALL | re.IGNORECASE)

    # Limpieza adicional
    texto_limpio = re.sub(r'Suscríbete a nuestro WhatsApp y recibe las noticias todos los días\.', '', texto_limpio, flags=re.DOTALL)
    texto_limpio = texto_limpio.strip()

    return texto_limpio, tags

# Aplicar la función y crear las nuevas columnas
letra = letra.copy()
letra[['articulo', 'autor']] = pd.DataFrame(letra['articulo'].apply(procesar_articulo).tolist(), index=letra.index)

# Reordenar columnas (opcional)
letra = letra[['titulo', 'articulo', 'autor', 'fecha', 'url']]
# Guardar el resultado
#letra.to_csv('letradelsur_limpio_con_tags.csv', index=False)

In [ ]:
def extraer_lugar(texto):
    # Patrón más flexible para nombres de ciudad (1-5 palabras) seguidas de "Jalisco.-"
    lugar_match = re.search(r'((?:\b[\wáéíóúüñÁÉÍÓÚÜÑ]+\b\s*){1,5}, Jalisco\.-)', texto)
    if lugar_match:
        lugar = lugar_match.group(1).strip()
        # Eliminar ".-" al final y cualquier espacio sobrante
        lugar = re.sub(r'\.-\s*$', '', lugar)
        texto_limpio = texto.replace(lugar_match.group(0), '', 1).strip()
        # Limpieza adicional: eliminar espacios dobles que puedan quedar
        texto_limpio = re.sub(r'\s+', ' ', texto_limpio).strip()
    else:
        lugar = ""
        texto_limpio = texto
    return texto_limpio, lugar

# Aplicar la función para extraer lugares
letra[['articulo', 'lugar']] = pd.DataFrame(letra['articulo'].apply(extraer_lugar).tolist(), index=letra.index)


# Reordenar columnas (opcional)
column_order = ['titulo', 'articulo', 'lugar', 'autor', 'fecha', 'url']
letra = letra[column_order]

# Guardar el resultado
letra.to_csv('cleaned_letrafria.csv', index=False, encoding='utf-8-sig')